In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats
from scipy.stats import uniform

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('D:\File_Analisa_Python\Analisa_Kredit.csv', sep='|')

#df = df.drop(index=2044850)
#df['KOLTYPE'] = ['PL' if x in (1,2) else 'NPL' for x in df['KOLJUNI']]

#df = df.rename(columns={'KOLJUNI' : 'KOLEKTIBILITAS'})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2063530 entries, 0 to 2063529
Data columns (total 17 columns):
 #   Column          Dtype  
---  ------          -----  
 0   POSISI          object 
 1   ROA             float64
 2   CBAL            float64
 3   GAJI_TOTAL      float64
 4   ORGAMT          float64
 5   JUMLAH_CICILAN  float64
 6   KOL             float64
 7   JNS_KRD         object 
 8   JENIS           float64
 9   LONDSC          object 
 10  SEGMEN          object 
 11  RASIO           float64
 12  STATREK         float64
 13  STATKOL         float64
 14  TAHUN           float64
 15  BULAN           object 
 16  BLN             float64
dtypes: float64(12), object(5)
memory usage: 267.6+ MB


In [30]:
ml_test = df[['CBAL','GAJI_TOTAL','ORGAMT','JUMLAH_CICILAN','KOL','STATKOL','RASIO']]

ml_test = ml_test.drop(index=2063529)

ml_test

,CBAL,GAJI_TOTAL,ORGAMT,JUMLAH_CICILAN,KOL,STATKOL,RASIO
0,21585927.0,2045000.0,70000000.0,1137300.0,1.0,1.0,0.556136
1,2500000.0,1029000.0,30000000.0,725000.0,1.0,1.0,0.704567
2,88703447.0,2000000.0,138000000.0,1813300.0,1.0,1.0,0.906650
3,428918782.0,860000.0,557000000.0,7146800.0,1.0,1.0,8.310232
4,114635243.0,2870200.0,518000000.0,8392000.0,1.0,1.0,2.923838
...,...,...,...,...,...,...,...
2063524,162320000.0,NaN,162320000.0,1303200.0,1.0,1.0,0.000000
2063525,703000000.0,NaN,703000000.0,5007700.0,1.0,1.0,0.000000
2063526,156380000.0,NaN,156380000.0,1045700.0,1.0,1.0,0.000000
2063527,100000000.0,NaN,100000000.0,3042200.0,1.0,1.0,0.000000


In [61]:
ml_test = ml_test.fillna(0)

In [62]:
fitur = ml_test.drop(columns='STATKOL')
target = ml_test['STATKOL']

In [63]:
display(fitur.head(5))
display(target.head(5))

,CBAL,GAJI_TOTAL,ORGAMT,JUMLAH_CICILAN,KOL,RASIO
0,21585927.0,2045000.0,70000000.0,1137300.0,1.0,0.556136
1,2500000.0,1029000.0,30000000.0,725000.0,1.0,0.704567
2,88703447.0,2000000.0,138000000.0,1813300.0,1.0,0.906650
3,428918782.0,860000.0,557000000.0,7146800.0,1.0,8.310232
4,114635243.0,2870200.0,518000000.0,8392000.0,1.0,2.923838


0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: STATKOL, dtype: float64

In [64]:
model_LR = LogisticRegression()
model_LR.fit(fitur, target)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [65]:
x = fitur
y = target # target dari dataset
y_pred = model_LR.predict(x) # target dari hasil prediksi

print(classification_report(y, y_pred))

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         1.0       0.93      1.00      0.96   1915566
         2.0       0.00      0.00      0.00    147963

    accuracy                           0.93   2063529
   macro avg       0.46      0.50      0.48   2063529
weighted avg       0.86      0.93      0.89   2063529



c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [67]:
ml_test[ml_test['STATKOL'] == 2]

,CBAL,GAJI_TOTAL,ORGAMT,JUMLAH_CICILAN,KOL,STATKOL,RASIO
13,5.262094e+06,1513033.0,1.650000e+07,2.225000e+05,5.0,2.0,0.147055
14,9.543600e+06,759000.0,1.100000e+07,1.523000e+05,5.0,2.0,0.200658
15,0.000000e+00,2300000.0,3.085000e+07,4.956000e+05,5.0,2.0,0.215478
17,8.761314e+07,3400000.0,1.830000e+08,2.421800e+06,3.0,2.0,0.712294
28,0.000000e+00,2500000.0,4.500000e+07,5.933000e+05,5.0,2.0,0.237320
...,...,...,...,...,...,...,...
2063356,1.031820e+09,0.0,1.700000e+09,1.700000e+09,5.0,2.0,0.000000
2063361,1.287854e+07,2000000.0,4.500000e+07,7.567000e+05,5.0,2.0,0.378350
2063363,5.104874e+06,0.0,6.000000e+06,6.021000e+04,5.0,2.0,0.000000
2063367,1.863055e+07,0.0,1.920000e+07,2.541500e+05,5.0,2.0,0.000000
